In [1]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
import shutil

from sklearn.model_selection import StratifiedKFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data

In [2]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [3]:
# データフレームをyolo形式に変換
def convert_to_yolo(df: pd.DataFrame) -> pd.DataFrame:
    bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        df[column] = bboxs[:,i]
    df.drop(columns=['bbox'], inplace=True)
    df['x_center'] = df['x'] + df['w']/2
    df['y_center'] = df['y'] + df['h']/2
    df['classes'] = 0
    return df

In [4]:
df = convert_to_yolo(df)
df.head()

,image_id,width,height,source,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0,88.0,202.5,0


In [5]:
# データを分割 
def stratified_kfold_split_df(df: pd.DataFrame) -> pd.DataFrame:
    fold_id = np.zeros((df.shape[0],1))
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    # sourceでの層化
    for (ff, (train_index, test_index)) in enumerate(skf.split(df, df['source'])):
        fold_id[test_index]= int(ff)

    df['fold'] = fold_id.copy()
    df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes', 'fold']]
    return df

In [6]:
df = stratified_kfold_split_df(df)
df.head()

,image_id,x,y,w,h,x_center,y_center,classes,fold
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0,0.0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0,2.0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0,3.0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0,3.0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0,4.0


In [7]:
def create_labels_and_copy_images(df: pd.DataFrame, fold: int, source: str = "train"):
    val_index = set(df[df['fold'] == fold]['image_id'])

    # 画像ごとにバウンディングボックスをループ処理
    for name,mini in tqdm(df.groupby('image_id')):    
        # ファイルの保存場所
        if name in val_index:
            path2save = 'valid/'
        else:
            path2save = 'train/'   
        # ラベルのストレージパス
        if not os.path.exists(f'convertor/fold{fold}/labels/' + path2save):
            os.makedirs(f'convertor/fold{fold}/labels/' + path2save)
        with open(f'convertor/fold{fold}/labels/' + path2save + name + ".txt", 'w+') as f:
            # Yolo形式の要件に従って座標を正規化
            row = mini[['classes','x_center','y_center','w','h']].astype(float).values
            row = row / 1024
            row = row.astype(str)
            for j in range(len(row)):
                text = ' '.join(row[j])
                f.write(text)
                f.write("\n")
        if not os.path.exists(f'convertor/fold{fold}/images/{path2save}'):
            os.makedirs(f'convertor/fold{fold}/images/{path2save}')
        # 画像の前処理は不要 => それらをバッチとしてコピー
        shutil.copy(f"../input/global-wheat-detection/{source}/{name}.jpg", f'convertor/fold{fold}/images/{path2save}/{name}.jpg')

In [8]:
create_labels_and_copy_images(df, fold=2, source="train")

100%|██████████| 3373/3373 [00:44<00:00, 76.39it/s]


In [9]:
!git clone https://github.com/ultralytics/yolov5  && cd yolov5 && pip install -r requirements.txt &> /dev/null

Cloning into 'yolov5'...
remote: Enumerating objects: 15943, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 15943 (delta 28), reused 39 (delta 16), pack-reused 15880
Receiving objects: 100% (15943/15943), 14.61 MiB | 25.44 MiB/s, done.
Resolving deltas: 100% (10927/10927), done.


In [10]:
# 割り当てられたGPUを確認
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
    
else:
    print(gpu_info)

Sat Sep  9 10:03:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

# YAML設定ファイル

In [11]:
yaml_text = """train: /kaggle/working/convertor/fold2/images/train/
val: /kaggle/working/convertor/fold2/images/valid/

nc: 1
names: ['wheat']"""

In [12]:
with open("wheat.yaml", 'w') as f:
    f.write(yaml_text)
%cat wheat.yaml

train: /kaggle/working/convertor/fold2/images/train/
val: /kaggle/working/convertor/fold2/images/valid/

nc: 1
names: ['wheat']

# Training

In [13]:
!python ./yolov5/train.py --img 512 --batch 2 --epochs 3 --workers 2 --data wheat.yaml --cfg "./yolov5/models/yolov5s.yaml" --name yolov5x_fold2 --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=wheat.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=2, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=yolov5x_fold2, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest

In [14]:
!ls ./yolov5/runs/train/yolov5x_fold2/weights/ -lh

total 28M
-rw-r--r-- 1 root root 14M Sep  9 10:10 best.pt
-rw-r--r-- 1 root root 14M Sep  9 10:10 last.pt


# Prediction

In [15]:
!ls /kaggle/input/global-wheat-detection/test

2fd875eaa.jpg  51b3e36ab.jpg  53f253011.jpg  aac893a91.jpg  cc3532ff6.jpg
348a992bb.jpg  51f1be19e.jpg  796707dd7.jpg  cb8d261a3.jpg  f5a1f0358.jpg


In [16]:
!python ./yolov5/detect.py --weights ./yolov5/runs/train/yolov5x_fold2/weights/best.pt --img 512 --conf 0.1 --source /kaggle/input/global-wheat-detection/test --save-txt --save-conf --exist-ok

detect: weights=['./yolov5/runs/train/yolov5x_fold2/weights/best.pt'], source=/kaggle/input/global-wheat-detection/test, data=yolov5/data/coco128.yaml, imgsz=[512, 512], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-215-ga6659d0 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/10 /kaggle/input/global-wheat-detection/test/2fd875eaa.jpg: 512x512 3 wheats, 7.6ms
image 2/10 /kaggle/input/global-wheat-detection/test/348a992bb.jpg: 512x512 3 wheats, 7.7ms
image 3/10 /kaggle/input/global-wheat-detection/test/51b3e36ab.jpg: 512x512 4 wheats, 7.7ms
image 4

In [17]:
!ls ./yolov5/runs/detect/exp/labels/

2fd875eaa.txt  51b3e36ab.txt  796707dd7.txt  cb8d261a3.txt  f5a1f0358.txt
348a992bb.txt  53f253011.txt  aac893a91.txt  cc3532ff6.txt


In [18]:
def convert_yolo_to_coco(s):
    x = int(1024 * (s[1] - s[3]/2))
    y = int(1024 * (s[2] - s[4]/2))
    w = int(1024 * s[3])
    h = int(1024 * s[4])
    
    return(str(s[5]) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h))

In [19]:
with open('submission.csv', 'w') as myfile:

    # prepare submission
    wfolder = './yolov5/runs/detect/exp/labels/'
    for f in os.listdir(wfolder):
        fname = wfolder + f
        xdat = pd.read_csv(fname, sep = ' ', header = None)
        outline = f[:-4] + ' ' + ' '.join(list(xdat.apply(lambda s: convert_yolo_to_coco(s), axis = 1)))
        myfile.write(outline + '\n')
        
myfile.close()   

In [20]:
!cat submission.csv

cb8d261a3 0.101735 0 0 872 641
aac893a91 0.100034 363 322 542 300 0.157436 343 408 300 282
53f253011 0.105347 450 486 108 72 0.118474 46 0 254 183 0.134971 545 413 60 126 0.159892 538 494 75 76
cc3532ff6 0.106505 637 871 197 152 0.107327 419 857 252 144 0.112828 242 510 467 46 0.120426 176 532 211 144 0.131002 74 577 412 167 0.164819 272 679 387 270 0.170624 828 610 195 80
f5a1f0358 0.127284 209 88 160 17 0.190621 212 76 190 40 0.193568 231 68 152 19
51b3e36ab 0.106601 444 425 106 133 0.110323 444 510 72 81 0.120233 228 383 485 572 0.125546 146 587 670 311
796707dd7 0.104385 387 452 323 57 0.110018 374 0 218 1 0.200723 694 130 218 108
348a992bb 0.108021 342 440 332 94 0.109444 683 430 339 120 0.12174 313 478 350 120
2fd875eaa 0.111721 434 99 186 924 0.114829 467 432 167 200 0.11612 854 471 168 245
